### This is a program to simulate tree based Pseudo-Least Recently Used(PLRU) replacement policy for an n-way set assoiative  cache

In [1]:
import math
import numpy as np

In [2]:
# create a node class for the bit tree that is maintained for each set
class Node(object):  
    def __init__(self):
        self.bit = 0
        self.left = None
        self.right = None
        self.leaf = False

##### Create a root node
root = Node()
##### Create a tree with root as root node
tree = Tree(root)
##### Build tree with height = logbase2 (n-ways) by calling build_tree function
tree.build_tree(root,4)

In [3]:
class Tree(object): 
    def __init__(self,root):
        self.root = root
        self.height = 0
    
    def insert(self, root, height):
        it = height-1
        if it == 0:
            root.right = []
            root.left = []
            root.leaf = True
            return
        root.left = Node()
        root.right = Node()
        self.insert(root.left,it)
        self.insert(root.right,it)
    
    def build_tree(self, root, ways):
        self.height = int(np.log2(ways))
        self.insert(root, self.height)
        return root

##### build cache args: cache_size in KB, no of words per line, no of ways
cache = CacheBuilder(4,16,4)
4 = 4KB of cache,   16 = 16 Words per cache line(64 bytes), 4 = no of ways
##### create cache by calling create_cache() method
cache.create_cache()
##### access cache through the cache_dict dictionary. 
cache.cache_dict
The dictionary has all sets and each set has one tree. Each leaf node of tree is linked to two lines of the set

In [4]:
class CacheBuilder(object):
    # 1 word = 4bytes = 32bits
    # cache size in KB (kilobytes)
    # n = no of ways
    # line_size = no of words in one line 
    # i.e line_size is the max no of intigers that can be in one line 
    # assuming one int = 32 bits or 4 bytes or 1 word
    def __init__(self, cache_size, line_size, n):
        self.ways = n
        self.line_size = line_size
        self.cache_size = cache_size
        self.no_of_lines = (cache_size*1024)/(line_size*4)
        self.no_of_int = self.linesize/1
        if (n<=self.no_of_lines):
            self.no_of_sets = self.no_of_lines/n
        else:
            self.no_of_sets = 0
            print "no of ways cannot be less then total no of lines"
        self.cache_dict = {}
        
    def append_lines(self,lines,root):
        if root.leaf == True:
            lines.append(root.left)
            lines.append(root.right)
            return lines
        self.append_lines(lines,root.left)
        self.append_lines(lines,root.right)    
        return lines
    
    def create_cache(self):
        if(self.no_of_sets == 0):
            print "cache cannot be built, reason: no of sets cannot be less then no of lines"
            return
        # create a tree for each set
        for i in range(self.no_of_sets):
            self.cache_dict[i] = {}
            root = Node()
            tree = Tree(root)
            tree.build_tree(root, self.ways)
            self.cache_dict[i]["tree"] = tree
            #create lines
            lines = []
            self.cache_dict[i]["lines"] = self.append_lines(lines,tree.root)
            self.cache_dict[i]["tag"] = []
            for j in range(self.ways):
                self.cache_dict[i]["tag"].append("")
            
    def lookup(self, set_no, tag):
        if tag in self.cache_dict[set_no]["tag"]:
            return self.cache_dict[set_no]["lines"][self.cache_dict[set_no]["tag"].index(tag)]
            #return True
        else:
            return None
    
    def on_miss_load(self, element, set_no, tag):
        # load the element in the memory by PLRU algorith
        rt = self.cache_dict[set_no]["tree"].root
        while rt.leaf == False:
            if rt.bit == 0:
                rt.bit = 1
                rt = rt.left
            else:
                rt.bit = 0
                rt = rt.right
        if rt.bit == 0:
            rt.bit = 1
            if len(rt.left) > 0:
                rt.left[:] = []
            for num in element:
                rt.left.append(num)
        else:
            rt.bit = 0
            if len(rt.right) > 0:
                rt.right[:] = []
            for num in element:
                rt.right.append(num)
        # updata the tag field for that loaded element
        self.cache_dict[set_no]["tag"][self.cache_dict[set_no]["lines"].index(element)] = tag
        

##### create a Memory block 
mem = MemBuilder(MS, cache.line_size, cache.no_of_sets)

##### multiply the two matrices
res = mem.multiply()

In [293]:
class MemBuilder(object):
    # build the memory
    # memory is a 2d array column = line_size
    # rows = MS/line_size
    # line size is the number of words (in our case 1 int is one word) per line
    # no_of_sets = no of sets in 
    def __init__(self, MS, line_size, no_of_sets):
        self.mat_size = MS
        self.line_size = line_size
        self.no_of_sets = no_of_sets
        self.mem = []
        self.mat1 = []
        self.mat2 = []
        self.hit = 0
        self.miss = 0
        self.add_lookup = {}
        counter = 1
        # fill the memory here
        for i in range(((2*MS*MS)/line_size)):
            line = []
            for j in range(line_size):
                self.add_lookup[counter] = {}
                self.add_lookup[counter]["set"] = len(self.mem)%no_of_sets
                self.add_lookup[counter]["tag"] = len(self.mem)/no_of_sets
                line.append(counter)
                if counter <= (MS*MS):
                    self.mat1.append(counter)
                else:
                    self.mat2.append(counter)
                counter+=1
            self.mem.append(line)
        # build two matrices of size MS*MS
        arr1 = np.array(self.mat1).reshape(MS,MS)
        arr2 = np.array(self.mat2).reshape(MS,MS)
        self.mat1 = arr1.tolist()
        self.mat2 = arr2.tolist()
                
    # multiply both the matrix
    def multiply(self,cache):
        res = []
        for i in range (self.mat_size):
            res_row = []
            for j in range (self.mat_size):
                res_sum = 0
                for x in range(self.mat_size):
                    
                    # check if cache has the elements
                    set_no = self.add_lookup[self.mat1[i][x]]["set"]
                    tag = self.add_lookup[self.mat1[i][x]]["tag"]
                    lookup = cache.lookup(set_no, tag)
                    if lookup != None:
                        #print "was looking for ", self.mat1[i][x], "found in ", lookup
                        self.hit += 1
                    else:
                        #print "was looking for", self.mat1[i][x], "could not find in cache"
                        cache.on_miss_load(self.mem[(tag*self.no_of_sets)+set_no], set_no, tag)
                        self.miss +=1
                    set_no = self.add_lookup[self.mat2[i][x]]["set"]
                    tag = self.add_lookup[self.mat2[i][x]]["tag"]
                    lookup = cache.lookup(set_no, tag)
                    if lookup != None:
                        #print "was looking for ", self.mat2[i][x], "found in ", lookup
                        self.hit += 1
                    else:
                        self.miss +=1
                        #print "was looking for", self.mat2[i][x], "could not find in cache"
                        cache.on_miss_load(self.mem[(tag*self.no_of_sets)+set_no], set_no, tag)
                    # resume operation
                    
                    res_sum = res_sum + (self.mat1[i][x]*self.mat2[x][j])
                res_row.append(res_sum)
            res.append(res_row)
        return res

In [312]:
cache = CacheBuilder(4,16,4)
cache.create_cache()

In [313]:
cache.cache_dict

{0: {'lines': [[], [], [], []],
  'tag': ['', '', '', ''],
  'tree': <__main__.Tree at 0x7f1ae84f1bd0>},
 1: {'lines': [[], [], [], []],
  'tag': ['', '', '', ''],
  'tree': <__main__.Tree at 0x7f1ae84f1b90>},
 2: {'lines': [[], [], [], []],
  'tag': ['', '', '', ''],
  'tree': <__main__.Tree at 0x7f1ae84b1850>},
 3: {'lines': [[], [], [], []],
  'tag': ['', '', '', ''],
  'tree': <__main__.Tree at 0x7f1ae84b1410>},
 4: {'lines': [[], [], [], []],
  'tag': ['', '', '', ''],
  'tree': <__main__.Tree at 0x7f1ae84b1650>},
 5: {'lines': [[], [], [], []],
  'tag': ['', '', '', ''],
  'tree': <__main__.Tree at 0x7f1ae84b1890>},
 6: {'lines': [[], [], [], []],
  'tag': ['', '', '', ''],
  'tree': <__main__.Tree at 0x7f1ae84b12d0>},
 7: {'lines': [[], [], [], []],
  'tag': ['', '', '', ''],
  'tree': <__main__.Tree at 0x7f1ae84b1990>},
 8: {'lines': [[], [], [], []],
  'tag': ['', '', '', ''],
  'tree': <__main__.Tree at 0x7f1ae84b1a90>},
 9: {'lines': [[], [], [], []],
  'tag': ['', '', '', '

In [314]:
mem = MemBuilder(100,cache.line_size,cache.no_of_sets)

In [315]:
res = mem.multiply(cache)

In [316]:
cache.cache_dict

{0: {'lines': [[9729,
    9730,
    9731,
    9732,
    9733,
    9734,
    9735,
    9736,
    9737,
    9738,
    9739,
    9740,
    9741,
    9742,
    9743,
    9744],
   [9985,
    9986,
    9987,
    9988,
    9989,
    9990,
    9991,
    9992,
    9993,
    9994,
    9995,
    9996,
    9997,
    9998,
    9999,
    10000],
   [19969,
    19970,
    19971,
    19972,
    19973,
    19974,
    19975,
    19976,
    19977,
    19978,
    19979,
    19980,
    19981,
    19982,
    19983,
    19984],
   [19713,
    19714,
    19715,
    19716,
    19717,
    19718,
    19719,
    19720,
    19721,
    19722,
    19723,
    19724,
    19725,
    19726,
    19727,
    19728]],
  'tag': [38, 39, 78, 77],
  'tree': <__main__.Tree at 0x7f1ae84f1bd0>},
 1: {'lines': [[19729,
    19730,
    19731,
    19732,
    19733,
    19734,
    19735,
    19736,
    19737,
    19738,
    19739,
    19740,
    19741,
    19742,
    19743,
    19744],
   [19985,
    19986,
    19987,
    19988,
    

In [317]:
print "hits: ",mem.hit
print "misses: ", mem.miss

hits:  1998750
misses:  1250


##### the misses increses as no of words per line (line_size) decreases. Assuming contigious allocation in memory